In [9]:
import pandas as pd
import numpy as np

# Chemins CSV (à modifier si besoin)
CSV_QUALITY_RAW = "../data/quality/quality-data.csv"
CSV_QUALITY_INTERPOLATED = "../data/quality/quality-data-interpolated.csv"
CSV_CRISE_COMPARAISON = "../data/quality/quality-data-with-crise.csv"

In [10]:
df = pd.read_csv(CSV_QUALITY_RAW)
df.columns = df.columns.str.strip()

def to_number(x):
    if pd.isna(x):
        return np.nan
    x = str(x).replace("\u202f", "").replace(" ", "")
    x = x.replace(",", ".")
    try:
        return float(x)
    except:
        return np.nan

for col in ["PLF", "CFX", "TOTAL"]:
    df[col] = df[col].apply(to_number)

df = df.sort_values(["INDICATEUR", "SOUS-INDICATEUR", "ANNEE"])

def interpolate_group(g):
    indicateur, sous_indicateur = g.name

    # --- Unité (on assume une seule unité par couple indicateur / sous-indicateur)
    unite = g["UNITE"].dropna().iloc[0] if "UNITE" in g.columns and g["UNITE"].notna().any() else np.nan

    # --- Proportion PLF dans (PLF+CFX) quand on l'a (pour répartir TOTAL si besoin)
    ratio = np.nan
    mask_known = g["PLF"].notna() & g["CFX"].notna() & ((g["PLF"] + g["CFX"]) > 0)
    if mask_known.any():
        ratio = (g.loc[mask_known, "PLF"] / (g.loc[mask_known, "PLF"] + g.loc[mask_known, "CFX"])).median()
    if np.isnan(ratio):
        ratio = 0.5  # fallback

    g = g.set_index("ANNEE")
    year_min = min(2011, g.index.min())
    years = range(year_min, g.index.max() + 2)
    g = g.reindex(years)

    g["INDICATEUR"] = indicateur
    g["SOUS-INDICATEUR"] = sous_indicateur
    g["UNITE"] = unite

    # 1) Si TOTAL existe et CFX manquant -> CFX = TOTAL - PLF
    m = g["TOTAL"].notna() & g["PLF"].notna() & g["CFX"].isna()
    g.loc[m, "CFX"] = g.loc[m, "TOTAL"] - g.loc[m, "PLF"]

    # 2) Si TOTAL existe et PLF manquant -> PLF = TOTAL - CFX
    m = g["TOTAL"].notna() & g["CFX"].notna() & g["PLF"].isna()
    g.loc[m, "PLF"] = g.loc[m, "TOTAL"] - g.loc[m, "CFX"]

    # 3) Si TOTAL existe et PLF+CFX manquent -> répartir via ratio
    m = g["TOTAL"].notna() & g["PLF"].isna() & g["CFX"].isna()
    g.loc[m, "PLF"] = g.loc[m, "TOTAL"] * ratio
    g.loc[m, "CFX"] = g.loc[m, "TOTAL"] * (1 - ratio)

    # 4) Interpolation linéaire
    g["PLF"] = g["PLF"].interpolate(method="linear", limit_direction="both")
    g["CFX"] = g["CFX"].interpolate(method="linear", limit_direction="both")

    # 5) Compléter TOTAL si manquant
    mt = g["TOTAL"].isna()
    g.loc[mt, "TOTAL"] = g.loc[mt, "PLF"] + g.loc[mt, "CFX"]

    # 6) Arrondir toutes les données numériques à 2 décimales
    g[["PLF", "CFX", "TOTAL"]] = g[["PLF", "CFX", "TOTAL"]].round(2)

    return g.reset_index().rename(columns={"index": "ANNEE"})

df_interpolated = (
    df.groupby(["INDICATEUR", "SOUS-INDICATEUR"], group_keys=False)
      .apply(interpolate_group)
)

print("NaN restants :\n", df_interpolated[["PLF","CFX","TOTAL"]].isna().sum())
df_interpolated.to_csv(CSV_QUALITY_INTERPOLATED, index=False)
print("Fichier sauvegardé ✅")

NaN restants :
 PLF      0
CFX      0
TOTAL    0
dtype: int64
Fichier sauvegardé ✅


C:\Users\User\AppData\Local\Temp\ipykernel_3000\2376604853.py:70: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(interpolate_group)


In [7]:
df = pd.read_csv(CSV_QUALITY_INTERPOLATED)

# --- 1) Dataset normal
df_normal = df.copy()
df_normal["MODE"] = "Normal"

# --- 2) NOUVELLE APPROCHE : Redistribution pour les données en %
# En crise, la qualité SE DÉGRADE (pas de multiplication simple)

# Définir les mappings de dégradation pour chaque indicateur
QUALITY_DEGRADATION = {
    "Excellent": {"Excellent": 0.50, "Tres bon": 0.30, "Bon": 0.15, "Moyen": 0.05, "Mauvais": 0.00},
    "Tres bon": {"Excellent": 0.00, "Tres bon": 0.40, "Bon": 0.40, "Moyen": 0.15, "Mauvais": 0.05},
    "Bon": {"Excellent": 0.00, "Tres bon": 0.10, "Bon": 0.40, "Moyen": 0.35, "Mauvais": 0.15},
    "Moyen": {"Excellent": 0.00, "Tres bon": 0.00, "Bon": 0.15, "Moyen": 0.45, "Mauvais": 0.40},
    "Mauvais": {"Excellent": 0.00, "Tres bon": 0.00, "Bon": 0.00, "Moyen": 0.20, "Mauvais": 0.80}
}

# --- 3) Identifier les indicateurs en %
# On traite différemment les indicateurs en % (distributions) vs valeurs absolues
def is_percentage_indicator(row):
    """Détermine si un indicateur est un pourcentage (distribution)"""
    # Vérifier si l'unité est '%'
    if pd.notna(row['UNITE']) and row['UNITE'] == '%':
        # Vérifier si c'est une distribution (Excellent, Bon, Mauvais, etc.)
        sous_indic = str(row['SOUS-INDICATEUR']).strip()
        if sous_indic in ["Excellent", "Tres bon", "Bon", "Moyen", "Mauvais"]:
            return True
    return False

# Marquer les lignes
df_normal['IS_PCT_DISTRIBUTION'] = df_normal.apply(is_percentage_indicator, axis=1)

# --- 4) Dataset crise avec logique adaptée
df_crise = df.copy()
df_crise["MODE"] = "Crise"
df_crise['IS_PCT_DISTRIBUTION'] = df_crise.apply(is_percentage_indicator, axis=1)

# Pour les pourcentages (distributions) : REDISTRIBUTION
# Pour les autres valeurs : MULTIPLICATION par coefficient

# A) Traiter les distributions en % (redistribution)
pct_mask = df_crise['IS_PCT_DISTRIBUTION']

if pct_mask.any():
    print(f"⚠️ {pct_mask.sum()} lignes détectées comme distributions en % → Redistribution appliquée")
    
    # Grouper par ANNEE + INDICATEUR pour redistribuer
    for (annee, indicateur), group in df_crise[pct_mask].groupby(['ANNEE', 'INDICATEUR']):
        indices = group.index
        
        # Créer une nouvelle distribution dégradée
        new_distribution = {}
        for _, row in group.iterrows():
            sous_indic = str(row['SOUS-INDICATEUR']).strip()
            if sous_indic in QUALITY_DEGRADATION:
                # Distribuer selon la matrice de dégradation
                for target, weight in QUALITY_DEGRADATION[sous_indic].items():
                    if target not in new_distribution:
                        new_distribution[target] = 0
                    # PLF et CFX traités séparément
                    new_distribution[target] += row['PLF'] * weight
        
        # Normaliser pour sommer à 100% (au cas où)
        total_new = sum(new_distribution.values())
        if total_new > 0:
            for key in new_distribution:
                new_distribution[key] = (new_distribution[key] / total_new) * 100
        
        # Appliquer les nouvelles valeurs
        for idx, row in group.iterrows():
            sous_indic = str(row['SOUS-INDICATEUR']).strip()
            if sous_indic in new_distribution:
                # Appliquer le même ratio PLF/CFX qu'en normal
                ratio_plf = row['PLF'] / row['TOTAL'] if row['TOTAL'] > 0 else 0.5
                df_crise.loc[idx, 'PLF'] = round(new_distribution[sous_indic] * ratio_plf, 2)
                df_crise.loc[idx, 'CFX'] = round(new_distribution[sous_indic] * (1 - ratio_plf), 2)
                df_crise.loc[idx, 'TOTAL'] = round(new_distribution[sous_indic], 2)

# B) Pour les autres indicateurs (valeurs absolues, scores, etc.) : MULTIPLICATION classique
non_pct_mask = ~df_crise['IS_PCT_DISTRIBUTION']

if non_pct_mask.any():
    print(f"✅ {non_pct_mask.sum()} lignes traitées avec coefficient ×1.70 (valeurs non-distributions)")
    COEF_CRISE_GLOBAL = 1.70
    df_crise.loc[non_pct_mask, "PLF"] = (df_crise.loc[non_pct_mask, "PLF"] * COEF_CRISE_GLOBAL).round(2)
    df_crise.loc[non_pct_mask, "CFX"] = (df_crise.loc[non_pct_mask, "CFX"] * COEF_CRISE_GLOBAL).round(2)
    df_crise.loc[non_pct_mask, "TOTAL"] = (df_crise.loc[non_pct_mask, "PLF"] + df_crise.loc[non_pct_mask, "CFX"]).round(2)

# --- 5) Calcul évolution (Crise vs Normal)
merge_cols = ["ANNEE", "INDICATEUR", "SOUS-INDICATEUR"]

df_compare = df_normal.merge(
    df_crise[merge_cols + ["PLF", "CFX", "TOTAL"]],
    on=merge_cols,
    suffixes=("_NORMAL", "_CRISE")
)

# Écart en valeur et variation en %
df_compare["ECART_TOTAL"] = (df_compare["TOTAL_CRISE"] - df_compare["TOTAL_NORMAL"]).round(2)
df_compare["VARIATION_PCT"] = ((df_compare["ECART_TOTAL"] / df_compare["TOTAL_NORMAL"]) * 100).round(2)

print("\n" + "="*80)
print("APERÇU DES DONNÉES (10 premières lignes)")
print("="*80)
print(df_compare.head(10))

# Vérification : somme des % par indicateur/année
print("\n" + "="*80)
print("VÉRIFICATION : Somme des distributions en % (doit = 100%)")
print("="*80)

check_pct = df_compare[df_compare['UNITE'] == '%'].copy()
if not check_pct.empty:
    check_pct['SOUS_CLEAN'] = check_pct['SOUS-INDICATEUR'].str.strip()
    
    # Filtrer uniquement les distributions (Excellent, Bon, etc.)
    quality_categories = ["Excellent", "Tres bon", "Bon", "Moyen", "Mauvais"]
    check_pct_dist = check_pct[check_pct['SOUS_CLEAN'].isin(quality_categories)]
    
    if not check_pct_dist.empty:
        sums_normal = check_pct_dist.groupby(['ANNEE', 'INDICATEUR'])['TOTAL_NORMAL'].sum()
        sums_crise = check_pct_dist.groupby(['ANNEE', 'INDICATEUR'])['TOTAL_CRISE'].sum()
        
        print("\nSOMMES MODE NORMAL (5 premiers):")
        print(sums_normal.head())
        print(f"\n→ Toutes les sommes = 100% ? {(sums_normal.round(0) == 100).all()}")
        
        print("\nSOMMES MODE CRISE (5 premiers):")
        print(sums_crise.head())
        print(f"→ Toutes les sommes = 100% ? {(sums_crise.round(0) == 100).all()}")

# Sauvegarde
df_compare.to_csv(CSV_CRISE_COMPARAISON, index=False)
print("\n" + "="*80)
print("✅ Fichier comparaison sauvegardé avec logique adaptée aux distributions !")
print("="*80)

   ANNEE                         INDICATEUR SOUS-INDICATEUR  PLF_NORMAL  \
0   2011  Confort et propreté de la chambre             Bon       20.60   
1   2012  Confort et propreté de la chambre             Bon       20.60   
2   2013  Confort et propreté de la chambre             Bon       20.60   
3   2011  Confort et propreté de la chambre       Excellent        8.75   
4   2012  Confort et propreté de la chambre       Excellent        8.75   
5   2013  Confort et propreté de la chambre       Excellent        8.75   
6   2011  Confort et propreté de la chambre         Mauvais        4.38   
7   2012  Confort et propreté de la chambre         Mauvais        4.38   
8   2013  Confort et propreté de la chambre         Mauvais        4.38   
9   2011  Confort et propreté de la chambre       Tres bon        15.00   

   CFX_NORMAL  TOTAL_NORMAL UNITE    MODE  PLF_CRISE  CFX_CRISE  TOTAL_CRISE  \
0       20.60         41.20     %  Normal      35.02      35.02        70.04   
1       20.60 